# Create Database And Tables

Create database and split csv into tables.

In [75]:
#Import Packages and Modules
import pandas as pd
from sqlalchemy import create_engine

In [76]:
df = pd.read_csv('RealEstate_California.csv')

In [77]:
engine = create_engine("sqlite:///Fission.db", echo=False)
df.rename(columns={'livingArea': 'livingSqFt', 'datePostedString': 'date'}, inplace=True)

print(df.head(5))

   Unnamed: 0                id  stateId  countyId  cityId country  \
0           0  95717-2087851113        9        77   24895     USA   
1           1    94564-18496265        9       189   36958     USA   
2           2    94564-18484475        9       190   36958     USA   
3           3    94564-18494835        9       191   36958     USA   
4           4  94564-2069722747        9       192   36958     USA   

         date  is_bankOwned  is_forAuction            event  ...  parking  \
0  2021-01-13             0              0  Listed for sale  ...        0   
1  2021-07-12             0              0  Listed for sale  ...        1   
2  2021-07-08             0              0  Listed for sale  ...        1   
3  2021-07-07             0              0  Listed for sale  ...        1   
4  2021-07-07             0              0  Listed for sale  ...        0   

   garageSpaces  hasGarage     levels pool  spa isNewConstruction  \
0           0.0          0          0    0    0

In [78]:
sum_of_nulls = df.isnull().sum().sum()
print("Total Null Values: ", sum_of_nulls)

Total Null Values:  596


In [79]:
df.dropna(axis=0, how='any', inplace=True)
sum_of_nulls = df.isnull().sum().sum()
print("Total Null Values: ", sum_of_nulls)


Total Null Values:  0


In [80]:
df['city'] = df.city.astype(str).str.upper()
df['county'] = df.county.astype(str).str.upper()
df['zipcode'] = df.zipcode.astype(int)

In [81]:
df.drop_duplicates(subset=['id'], inplace=True, keep="first")
clean_df = df.drop_duplicates(subset=['id'], inplace=True, keep="first")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30753 entries, 0 to 35388
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          30753 non-null  int64  
 1   id                  30753 non-null  object 
 2   stateId             30753 non-null  int64  
 3   countyId            30753 non-null  int64  
 4   cityId              30753 non-null  int64  
 5   country             30753 non-null  object 
 6   date                30753 non-null  object 
 7   is_bankOwned        30753 non-null  int64  
 8   is_forAuction       30753 non-null  int64  
 9   event               30753 non-null  object 
 10  time                30753 non-null  float64
 11  price               30753 non-null  float64
 12  pricePerSquareFoot  30753 non-null  float64
 13  city                30753 non-null  object 
 14  state               30753 non-null  object 
 15  yearBuilt           30753 non-null  int64  
 16  stre

In [82]:

hometype_list = df['homeType'].unique().tolist()
hometype_list = pd.DataFrame(hometype_list, columns=['homeType'])
hometype_list.to_sql('home_type', con=engine, if_exists='replace', index_label="hometype_index")


6

In [83]:
city_list = df['city'].unique().tolist()
city_list = pd.DataFrame(city_list, columns=['city'])
city_list.to_sql('city', con=engine, if_exists='replace', index_label="city_index")

1184

In [84]:
price_list = df.drop_duplicates(subset=['price', 'pricePerSquareFoot'], inplace=True)
price_list = df[['price', 'pricePerSquareFoot']]
price_list.to_sql('price', con=engine, if_exists='replace', index_label="price_index")


24545

In [85]:
zipcode_list = df['zipcode'].unique().tolist()
zipcode_list = pd.DataFrame(zipcode_list, columns=['zipcode'])
# zipcode_list.info()
zipcode_list.to_sql('zipcode', con=engine, if_exists='replace', index_label="zip_index")

1540

In [101]:
county_list = df.drop_duplicates(subset=['county'], keep="first")
county_list = pd.DataFrame(county_list, columns=['county'])
county_list.to_sql('county', con=engine, if_exists='replace', index_label="county_index")

58

In [102]:
zipcodes = pd.read_sql_table('zipcode', con = engine)
cities = pd.read_sql_table('city', con = engine)
home_types = pd.read_sql_table('home_type', con = engine)
counties = pd.read_sql_table('county', con = engine)
prices = pd.read_sql_table('price', con=engine)

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24545 entries, 0 to 35388
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          24545 non-null  int64  
 1   id                  24545 non-null  object 
 2   stateId             24545 non-null  int64  
 3   countyId            24545 non-null  int64  
 4   cityId              24545 non-null  int64  
 5   country             24545 non-null  object 
 6   date                24545 non-null  object 
 7   is_bankOwned        24545 non-null  int64  
 8   is_forAuction       24545 non-null  int64  
 9   event               24545 non-null  object 
 10  time                24545 non-null  float64
 11  price               24545 non-null  float64
 12  pricePerSquareFoot  24545 non-null  float64
 13  city                24545 non-null  object 
 14  state               24545 non-null  object 
 15  yearBuilt           24545 non-null  int64  
 16  stre

In [128]:
geo = df.merge(zipcodes, on=['zipcode'])\
    .merge(cities, how='left', on=['city'])\
    .merge(home_types, how='left', on=['homeType'])\
    .merge(counties, how='right', on=['county'])

counties.info()
geo.info()
geo_list = geo[['zip_index', 'city_index', 'county_index']].drop_duplicates()
# geo_list.rename(columns={'county_index': 'countyId'}, inplace=True)

geo_list.to_sql('geography', con=engine, if_exists='replace', index_label="geo_index")

# geo_list.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   county_index  58 non-null     int64 
 1   county        58 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24545 entries, 0 to 24544
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          24545 non-null  int64  
 1   id                  24545 non-null  object 
 2   stateId             24545 non-null  int64  
 3   countyId            24545 non-null  int64  
 4   cityId              24545 non-null  int64  
 5   country             24545 non-null  object 
 6   date                24545 non-null  object 
 7   is_bankOwned        24545 non-null  int64  
 8   is_forAuction       24545 non-null  int64  
 9   event               24545 n

1800

In [187]:

geo_ids = pd.read_sql_table('geography', con = engine)
# df.info()

geo_ids = geo_ids.merge(counties, on=['county_index'])\
        .merge(zipcodes, on=['zip_index'])\
        .merge(cities, on=['city_index'])\
        .merge(df, on=['county', 'city', 'zipcode'])\
        .merge(prices, how='inner', on=['price', 'pricePerSquareFoot'])\
        .merge(home_types, on=['homeType'])


# geo_ids = geo_ids.geo_index.astype(int)
# geo_ids.info()
geo_ids.dropna(subset=['geo_index'])
property_df = geo_ids.drop(['stateId', 'cityId', 'country', 'time', 
        'time', 'longitude', 'latitude', 'currency', 
        'livingAreaValue', 'Unnamed: 0', 'isNewConstruction',
        'homeType', 'zipcode', 'county', 'city', 'state',
        'price', 'pricePerSquareFoot', 
        'city_index', 'zip_index', 'county_index', 'countyId'
        ], axis=1)
property_df.info()

property_df.to_sql('property_info', con=engine, if_exists='replace', index_label="property_index", chunksize=500)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24545 entries, 0 to 24544
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   geo_index       24545 non-null  int64  
 1   id              24545 non-null  object 
 2   date            24545 non-null  object 
 3   is_bankOwned    24545 non-null  int64  
 4   is_forAuction   24545 non-null  int64  
 5   event           24545 non-null  object 
 6   yearBuilt       24545 non-null  int64  
 7   streetAddress   24545 non-null  object 
 8   hasBadGeocode   24545 non-null  int64  
 9   description     24545 non-null  object 
 10  livingSqFt      24545 non-null  float64
 11  lotAreaUnits    24545 non-null  object 
 12  bathrooms       24545 non-null  float64
 13  bedrooms        24545 non-null  float64
 14  buildingArea    24545 non-null  float64
 15  parking         24545 non-null  int64  
 16  garageSpaces    24545 non-null  float64
 17  hasGarage       24545 non-null 

24545

In [190]:
assert len(property_df['id'].unique())==property_df.shape[0]
assert property_df.groupby(['date','id']).size().max()==1